In [1]:
import warnings # type: ignore
warnings.filterwarnings('ignore') # type: ignore
from bs4 import BeautifulSoup # type: ignore
import os # type: ignore
import pandas as pd # type: ignore
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout # type: ignore
import time # type: ignore

SEASONS = list(range(2015, 2026))
DATA_DIR = 'downloads'

async def get_html(url, selector, sleep=5, retries=5):
    html = None
    for i in range(1, retries + 1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.webkit.launch()
                page = await browser.new_page()
                await page.goto(url)
                if 'College Basketball at Sports-Reference.com' in str(await page.title()) and '404 error' not in str(await page.title()):
                    print(await page.title())
                else:
                    return None
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

async def scrape_teams():
    teams = []
    url = 'https://www.sports-reference.com/cbb/schools/'
    html = await get_html(url, '#all_NCAAM_schools')
    soup = BeautifulSoup(html)
    links = soup.find_all('a')
    hrefs = [l.get('href') for l in links]
    for href in hrefs:
        if href is not None and 'schools' in href:
            teams.append(href.split('schools/')[-1].split('/men')[0])
    return teams

async def scrape_stats(season, team):
    url = f"https://www.sports-reference.com/cbb/schools/{team}/men/{season}.html"
    save_path = os.path.join(DATA_DIR, url.split('schools/')[-1].replace('/','_'))
    if not os.path.exists(save_path):
        html = await get_html(url, '#content')
        if html is not None:
            with open(save_path, 'w+') as f:
                f.write(html)

# TEAMS = await scrape_teams()
# for season in SEASONS:
#     for team in TEAMS:
#         await scrape_stats(season, team)

WINNERS = []
YEAR = 2024
ncaa_page = await get_html('https://www.sports-reference.com/cbb/postseason/', '#content')
soup = BeautifulSoup(ncaa_page)
table = soup.find('table', {'id':'ncaa-tournament-history_NCAAM'}).find('tbody')
rows = table.find_all('tr')
for row in rows[:-1]:
    columns = row.find_all('td')
    if len(columns) > 0:
        winner = columns[0].find('a').get('href').split('schools/')[-1].split('/men')[0]
        if YEAR == 2020: YEAR -= 1
        WINNERS.append(str(YEAR) + ' ' + winner)
        YEAR -= 1

####################################################
pd.set_option('display.max_columns', 10000)
team_pages = os.listdir(DATA_DIR)
team_pages = [os.path.join(DATA_DIR, f) for f in team_pages if f.endswith('.html')]

def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select('tr.over_header')]
    [s.decompose() for s in soup.select('tr.thead')]
    return soup

final_df = pd.DataFrame()
for i, team_page in enumerate(team_pages):
    try:
        soup = parse_html(team_page)
        STATS = ['season-total_per_game','conf_per_game','players_per_game','players_per_game_conf','players_advanced','players_advanced_conf']
        
        df = pd.DataFrame()
        for STAT in STATS:
            table = pd.read_html(str(soup), attrs={'id':STAT}, index_col=0)[0].iloc[:-1]
            table = table.apply(pd.to_numeric, errors='coerce')
            df = pd.concat([df, table.mean(), table.max()])
        df = df.T
        
        TEAM, YEAR = team_page.split('downloads/')[-1].split('_')[0], team_page.split('_')[-1].split('.html')[0]
        df['Team'] = TEAM
        df['Year'] = YEAR
        df['Winner/Target'] = 1 if YEAR + ' ' + TEAM in WINNERS else 0
        df.loc[df['Year'] == 2025, 'Winner/Target'] = 2
        
        final_df = pd.concat([final_df, df], ignore_index=True)
        if i % 300 == 0:
            print(f"{i} / {len(team_pages)}")
    except Exception as e:
        print(f"SKIPPED {team_page} because {e}")
final_df.to_csv('ncaa_stats.csv')
display(final_df)


NCAA Tournament History | College Basketball at Sports-Reference.com
0 / 4446
SKIPPED downloads/canisius_men_2010.html because Reindexing only valid with uniquely valued Index objects
SKIPPED downloads/montana-state_men_2010.html because Reindexing only valid with uniquely valued Index objects
SKIPPED downloads/portland-state_men_2010.html because Reindexing only valid with uniquely valued Index objects
SKIPPED downloads/western-carolina_men_2010.html because Reindexing only valid with uniquely valued Index objects
SKIPPED downloads/georgia-tech_men_2010.html because Reindexing only valid with uniquely valued Index objects
SKIPPED downloads/san-jose-state_men_2010.html because Reindexing only valid with uniquely valued Index objects
SKIPPED downloads/rice_men_2010.html because Reindexing only valid with uniquely valued Index objects
SKIPPED downloads/mississippi-state_men_2010.html because Reindexing only valid with uniquely valued Index objects
SKIPPED downloads/cincinnati_men_2010.ht

,G,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,G,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,G,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,G,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Player,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Player,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Player,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Player,Pos,G,GS,MP,PER,TS%,3PAr,FTr,PProd,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/40,OBPM,DBPM,BPM,Awards,Player,Pos,G,GS,MP,PER,TS%,3PAr,FTr,PProd,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/40,OBPM,DBPM,BPM,Awards,Player,Pos,G,GS,MP,PER,TS%,3PAr,FTr,PProd,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/40,Awards,Player,Pos,G,GS,MP,PER,TS%,3PAr,FTr,PProd,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/40,Awards,Team,Year,Winner/Target
0,30.0,40.0,24.25,57.60,0.4210,16.35,33.80,0.4855,7.85,23.85,0.3300,13.10,18.00,0.7295,9.95,26.15,36.15,12.90,7.25,3.00,14.20,16.80,69.45,30.0,40.0,25.6,57.6,0.444,17.3,34.2,0.519,8.2,24.2,0.340,14.4,20.1,0.740,10.4,27.6,38.0,13.9,7.4,3.1,14.7,18.0,73.8,18.0,40.0,24.05,58.05,0.4140,15.75,33.25,0.4755,8.30,24.85,0.3340,12.55,17.10,0.7345,8.50,24.50,33.00,12.75,7.20,3.05,13.60,16.25,68.95,18.0,40.0,26.2,58.3,0.449,17.0,34.2,0.527,9.2,26.0,0.353,14.2,19.6,0.745,9.4,26.7,36.1,14.8,7.3,3.2,14.0,17.6,75.7,NaN,NaN,25.538462,11.538462,16.015385,1.838462,4.569231,0.422615,0.592308,1.838462,0.271400,1.238462,2.723077,0.459615,0.469769,0.961538,1.276923,0.724167,0.630769,1.830769,2.461538,0.946154,0.592308,0.223077,1.200000,1.469231,5.207692,NaN,NaN,NaN,30.0,30.0,29.8,4.8,12.1,1.000,2.0,5.8,0.403,2.9,6.3,1.000,1.000,3.0,3.8,1.000,2.0,3.6,5.4,3.1,1.4,1.3,2.8,2.9,14.6,NaN,NaN,NaN,15.000000,6.923077,16.100000,1.746154,4.600000,0.385818,0.592308,1.892308,0.261700,1.153846,2.730769,0.413909,0.438364,0.861538,1.153846,0.717545,0.615385,1.800000,2.407692,0.861538,0.569231,0.238462,1.184615,1.415385,4.938462,NaN,NaN,NaN,18.0,18.0,32.3,5.7,13.9,0.533,2.2,6.7,0.385,3.4,7.4,0.533,0.533,3.1,4.0,0.889,1.8,4.0,5.2,2.6,1.3,1.4,2.7,2.9,16.6,NaN,NaN,NaN,25.538462,11.538462,461.538462,10.330769,0.543615,0.304385,0.459846,148.153846,3.900000,12.761538,8.230769,13.084615,1.961538,2.300000,23.600000,18.992308,0.100000,0.492308,0.592308,0.038231,-6.176923,-1.846154,-8.023077,NaN,NaN,NaN,30.0,30.0,895.0,18.4,1.000,0.663,3.000,411.0,10.8,19.8,15.2,40.3,4.1,10.2,62.3,28.0,0.9,1.0,1.9,0.108,-0.2,2.9,-0.8,NaN,NaN,NaN,15.000000,6.923077,276.923077,5.453846,0.471545,0.352909,0.249091,86.000000,3.946154,14.361538,8.961538,13.915385,1.584615,1.723077,32.276923,19.592308,0.107692,0.030769,0.138462,-0.052154,NaN,NaN,NaN,18.0,18.0,582.0,17.3,0.572,0.672,0.457,282.0,11.4,31.5,15.6,60.8,3.3,11.1,100.0,33.9,1.1,0.3,0.9,0.063,NaN,virginia-military-institute,2018,0
1,33.0,40.3,25.30,56.80,0.4465,18.50,36.95,0.4995,6.90,19.95,0.3450,13.45,18.80,0.7140,10.00,24.40,34.35,13.95,5.70,3.90,12.50,17.00,71.00,33.0,40.3,25.3,58.8,0.462,19.2,37.2,0.515,7.6,22.2,0.348,15.0,20.4,0.736,10.3,26.3,36.0,15.2,6.6,4.8,13.5,18.0,71.8,18.0,40.3,25.30,55.85,0.4530,18.65,35.90,0.5195,6.60,19.95,0.3335,13.10,18.20,0.7200,8.00,22.30,30.30,13.60,5.40,3.50,11.25,16.10,70.25,18.0,40.3,25.4,57.6,0.465,18.7,36.0,0.520,6.8,21.8,0.356,14.3,20.0,0.723,8.2,23.5,31.3,14.8,6.2,4.3,12.1,17.3,71.1,NaN,NaN,28.083333,13.750000,18.241667,2.183333,4.783333,0.421083,0.516667,1.600000,0.243500,1.641667,3.183333,0.515636,0.470167,1.275000,1.750000,0.678727,0.716667,2.175000,2.900000,1.725000,0.400000,0.408333,1.216667,1.416667,6.158333,NaN,NaN,NaN,33.0,32.0,28.7,5.0,9.2,0.582,1.8,5.4,0.462,

In [2]:
final_df[final_df['Winner/Target'] == 1]

,G,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,G,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,G,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,G,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Player,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Player,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Player,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Player,Pos,G,GS,MP,PER,TS%,3PAr,FTr,PProd,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/40,OBPM,DBPM,BPM,Awards,Player,Pos,G,GS,MP,PER,TS%,3PAr,FTr,PProd,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/40,OBPM,DBPM,BPM,Awards,Player,Pos,G,GS,MP,PER,TS%,3PAr,FTr,PProd,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/40,Awards,Player,Pos,G,GS,MP,PER,TS%,3PAr,FTr,PProd,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/40,Awards,Team,Year,Winner/Target
87,40.0,40.1,27.65,62.60,0.4395,20.10,40.95,0.4875,7.55,21.75,0.3475,14.80,20.80,0.7130,12.45,25.20,37.60,14.80,6.65,3.95,12.70,18.65,77.50,40.0,40.1,30.7,65.8,0.466,23.6,45.9,0.514,8.0,23.5,0.355,16.1,22.9,0.725,15.8,28.0,43.7,18.1,7.1,4.4,13.6,19.5,84.4,18.0,40.3,27.90,62.20,0.4475,19.85,39.95,0.4955,8.05,22.25,0.3620,13.25,18.85,0.7030,10.45,22.80,33.20,15.05,6.35,4.35,11.55,17.50,77.15,18.0,40.3,30.6,66.3,0.461,23.0,45.5,0.505,8.5,23.7,0.365,13.6,19.0,0.716,13.6,25.6,39.1,17.4,6.4,5.2,11.9,17.6,81.7,NaN,NaN,32.066667,13.333333,15.273333,2.233333,4.846667,0.407800,0.533333,1.506667,0.330900,1.726667,3.353333,0.437467,0.448000,1.193333,1.706667,0.702857,1.046667,2.013333,3.040000,1.413333,0.546667,0.260000,0.893333,1.353333,6.173333,NaN,NaN,NaN,40.0,40.0,32.0,6.6,14.9,0.576,2.6,7.1,0.500,5.2,9.3,0.576,0.576,2.9,3.7,1.000,3.8,5.7,9.5,3.7,1.4,1.2,1.9,3.1,18.3,NaN,NaN,NaN,14.666667,6.000000,14.766667,2.153333,4.733333,0.394615,0.533333,1.506667,0.212455,1.613333,3.226667,0.436846,0.432923,0.926667,1.360000,0.578308,0.973333,1.860000,2.840000,1.280000,0.460000,0.260000,0.800000,1.280000,5.793333,NaN,NaN,NaN,18.0,18.0,32.6,7.0,16.1,0.598,2.9,7.8,0.432,5.1,9.5,0.598,0.598,3.1,3.8,0.815,3.7,5.2,8.9,3.6,1.3,1.2,2.0,3.1,18.9,NaN,NaN,NaN,32.066667,13.333333,535.000000,15.740000,0.498467,0.244667,0.471000,221.400000,6.673333,14.900000,10.726667,12.166667,2.060000,1.606667,16.940000,17.600000,1.313333,0.946667,2.240000,0.144400,1.726667,3.680000,5.406667,NaN,NaN,NaN,40.0,40.0,1281.0,28.2,0.706,0.576,1.250,659.0,18.6,38.3,20.7,22.7,3.7,5.4,50.0,27.7,4.3,2.3,6.1,0.232,7.7,5.5,9.8,NaN,NaN,NaN,14.666667,6.000000,241.666667,12.333333,0.458385,0.278462,0.346692,97.600000,7.000000,15.466667,11.180000,11.406667,2.006667,2.106667,14.530769,16.346667,0.753333,0.113333,0.893333,0.075600,NaN,NaN,NaN,18.0,18.0,586.0,27.1,0.653,0.606,0.667,307.0,18.0,30.7,21.8,29.9,5.1,9.8,29.4,30.9,2.5,0.5,2.5,0.207,NaN,north-carolina,2017,1
485,40.0,40.4,26.55,60.05,0.4420,19.90,39.65,0.5000,6.75,20.40,0.3295,12.95,18.05,0.7180,10.85,24.65,35.50,13.55,6.40,3.75,12.40,16.65,72.75,40.0,40.4,28.6,60.3,0.478,21.5,40.0,0.536,7.2,21.0,0.361,13.9,19.4,0.719,11.0,26.9,37.9,15.3,6.5,4.3,12.7,17.4,78.2,18.0,40.8,26.25,60.70,0.4335,19.80,40.40,0.4920,6.40,20.30,0.3155,13.30,18.20,0.7285,10.10,23.60,33.70,13.30,6.30,3.80,11.35,17.10,72.10,18.0,40.8,26.9,63.0,0.461,19.8,42.2,0.514,7.1,20.8,0.356,14.3,19.6,0.729,10.2,26.1,36.3,14.4,7.2,4.1,12.7,18.3,75.1,NaN,NaN,31.428571,14.285714,15.864286,2.221429,4.678571,0.453000,0.571429,1.607143,0.273571,1.657143,3.057143,0.533692,0.515714,1.064286,1.492857,0.692167,0.750000,2.021429,2.764286,1.228571,0.492857,0.321429,0.921429,1.264286,6.057143,NaN,NaN,NaN,40.0,39.0,35.1,6.6,13.9,0.661,2.6,6.5,0.500,4.0,7.4,0